# Ax Service API with RayTune on PyTorch CNN

Ax integrates easily with different scheduling frameworks and distributed training frameworks. In this example, Ax-driven optimization is executed in a distributed fashion using [RayTune](https://ray.readthedocs.io/en/latest/tune.html). 

In [1]:
from ray import tune
from ray.tune import track
from ray.tune.suggest.ax import AxSearch

In [2]:
import torch
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate


init_notebook_plotting()

[INFO 06-18 16:58:11] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


## 1. Initialize client
Create a client object to interface with Ax. We specify `enforce_sequential_optimization` to be False, because Ray starts many trials in parallel and without disabling the sequential optimization enforcement, `AxClient` would expect the first few 5 to be completed with data before generating more trials.

When high parallelism is not required, it is best not to disable sequential optimization enforcement, as it allows the optimization to achieve optimal results in fewer (but sequential) trials. In cases where parallelism is more important, such as with discributed training using Ray, we choose to sacrifice minimizing resource utilization and run more trials in parallel. 

In [12]:
ax = AxClient(enforce_sequential_optimization=False)

## 2. Set up experiment
Here we set up the search space and specify the objective; refer to the Ax API tutorials for more detail.

In [5]:
ax.create_experiment(
    name="mnist_experiment",
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
    ],
    objective_name="mean_accuracy",
)

[INFO 06-18 16:58:11] ax.service.utils.dispatch: Using Bayesian Optimization generation strategy. Iterations after 5 will take longer to generate due to model-fitting.


## 3. Define how to evaluate trials
Since we use the Ax Service API here, we evaluate the parameterizations Ax suggests outside of Ax, using RayTune. The evaluation function follows its usual pattern, taking in a parameterization and outputting an objective value. For detail on evaluation functions, see [Trial Evaluation](https://ax.dev/docs/runner.html). 

In [6]:
def train_evaluate(parameterization):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, valid_loader, test_loader = load_mnist()
    net = train(train_loader=train_loader, parameters=parameterization, dtype=torch.float, device=device)
    track.log(
        mean_accuracy=evaluate(
            net=net,
            data_loader=valid_loader,
            dtype=torch.float,
            device=device,
        )
    )

## 4. Run optimization
Execute the Ax optimization and trial evaluation in RayTune using [AxSearch algorithm](https://ray.readthedocs.io/en/latest/tune-searchalg.html#ax-search):

In [7]:
tune.run(
    train_evaluate, 
    num_samples=40, 
    search_alg=AxSearch(ax),  # Note that the argument here is the `AxClient`.
    # To use GPU, specify: resources_per_trial={"gpu": 1}.
)

2019-06-18 16:58:11,606	WARNING worker.py:1331 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-06-18 16:58:11,608	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-18_16-58-11_607883_6566/logs.
2019-06-18 16:58:11,720	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:33441 to respond...
2019-06-18 16:58:11,850	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:63939 to respond...
2019-06-18 16:58:11,854	INFO services.py:806 -- Starting Redis shard with 3.44 GB max memory.
2019-06-18 16:58:11,887	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-18_16-58-11_607883_6566/logs.
2019-06-18 16:58:11,890	INFO services.py:1442 -- Starting the Plasma object store with 5.15 GB memory using /tmp.
2019-06-18 16:58:12,443	INFO function_runner.py:255 -- tun

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Memory usage on this node: 10.8/17.2 GB



2019-06-18 16:58:12,676	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling TensorBoard logging.
2019-06-18 16:58:12,677	WARNING logger.py:228 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2019-06-18 16:58:12,694	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling TensorBoard logging.
2019-06-18 16:58:12,695	WARNING logger.py:228 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2019-06-18 16:58:12,715	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling TensorBoard logging.
2019-06-18 16:58:12,716	WARNING logger.py:228 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2019-06-18 16:58:12,741	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling TensorBoard logging.
2019-06-18 16:58:12,743	WARNING logger.py:228 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2019-06-18 16:58:12,782	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling Tens

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs
Memory usage on this node: 10.8/17.2 GB
Result logdir: /Users/drfreund/ray_results/train_evaluate
Number of trials: 10 ({'RUNNING': 1, 'PENDING': 9})
PENDING trials:
 - train_evaluate_2_lr=0.10476,momentum=0.35651:	PENDING
 - train_evaluate_3_lr=7.7398e-06,momentum=0.78888:	PENDING
 - train_evaluate_4_lr=3.0642e-06,momentum=0.39297:	PENDING
 - train_evaluate_5_lr=0.064553,momentum=0.96158:	PENDING
 - train_evaluate_6_lr=0.0036458,momentum=0.043895:	PENDING
 - train_evaluate_7_lr=0.00027152,momentum=0.59981:	PENDING
 - train_evaluate_8_lr=0.00029511,momentum=0.25307:	PENDING
 - train_evaluate_9_lr=0.01379,momentum=0.82217:	PENDING
 - train_evaluate_10_lr=0.019602,momentum=0.18524:	PENDING
RUNNING trials:
 - train_evaluate_1_lr=0.0026454,momentum=0.64968:	RUNNING



2019-06-18 16:58:12,903	WARNING logger.py:134 -- Couldn't import TensorFlow - disabling TensorBoard logging.
2019-06-18 16:58:12,905	WARNING logger.py:228 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.


...

(pid=7293) 
(pid=7293) 25.3%
(pid=7293) 25.8%
(pid=7293) 26.3%
(pid=7293) 26.8%
(pid=7293) 27.3%
(pid=7293) 27.8%
(pid=7293) 28.3%
(pid=7293) 28.8%
(pid=7293) 29.3%
(pid=7293) 29.8%
(pid=7293) 30.3%
(pid=7293) 30.8%
(pid=7289) 
(pid=7289) 48.2%
(pid=7289) 48.7%
(pid=7289) 49.2%
(pid=7289) 49.7%
(pid=7289) 50.2%
(pid=7289) 50.7%
(pid=7289) 51.2%
(pid=7289) 51.7%
(pid=7289) 52.2%
(pid=7289) 52.7%
(pid=7289) 53.2%
(pid=7289) 53.7%
(pid=7289) 54.2%
(pid=7289) 54.7%
(pid=7289) 55.1%
(pid=7289) 55.6%
(pid=7289) 56.1%
(pid=7294) 
(pid=7294) 50.2%
(pid=7294) 50.7%
(pid=7294) 51.2%
(pid=7294) 51.7%
(pid=7294) 52.2%
(pid=7294) 52.7%
(pid=7294) 53.2%
(pid=7294) 53.7%
(pid=7294) 54.2%
(pid=7294) 54.7%
(pid=7294) 55.1%
(pid=7294) 55.6%
(pid=7294) 56.1%
(pid=7293) 
(pid=7293) 31.3%
(pid=7293) 31.8%
(pid=7293) 32.3%
(pid=7293) 32.8%
(pid=7293) 33.3%
(pid=7293) 33.8%
(pid=7293) 34.3%
(pid=7293) 34.8%
(pid=7293) 35.3%
(pid=7293) 35.8%
(pid=7293) 36.3%
(pid=7293) 36.8%
(pid=7293) 37.3%
(pid=7293) 37.8%


(pid=7293) Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
(pid=7289) Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
(pid=7289) Processing...
(pid=7293) 
(pid=7293) 80.0%
(pid=7293) 80.5%
(pid=7293) 81.0%
(pid=7293) 81.5%
(pid=7293) 82.0%
(pid=7293) 82.5%
(pid=7293) 83.0%
(pid=7293) 83.5%
(pid=7293) 84.0%
(pid=7293) 84.5%
(pid=7293) 85.0%
(pid=7293) 85.5%
(pid=7293) 86.0%
(pid=7293) 86.4%
(pid=7293) 86.9%
(pid=7293) 87.4%
(pid=7293) 87.9%
(pid=7293) 88.4%
(pid=7293) 88.9%
(pid=7293) 89.4%
(pid=7293) 89.9%
(pid=7293) 90.4%
(pid=7293) 90.9%
(pid=7293) 91.4%
(pid=7293) 91.9%
(pid=7293) 92.4%
(pid=7293) 92.9%
(pid=7293) 93.4%
(pid=7293) 93.9%
(pid=7293) 94.4%
(pid=7293) 94.9%
(pid=7293) 95.4%
(pid=7293) 95.9%
(pid=7293) 96.4%
(pid=7293) 96.9%
(pid=7293) 97.4%
(pid=7293) 97.9%
(pid=7293) 98.4%
(pid=7293) 98.9%
(pid=7293) 99.4%
(pid=7293) 99.9%
(pid=7293) 100.4%
(pid=7289) 
(pid=7289) 0.0%
(pid=7289) 180.4%
(pid=7294) 
(pid=7294) 0.0%
(pid=7294) 180.4%
(pid=7294) Downloading htt

2019-06-18 17:02:14,457	INFO ray_trial_executor.py:187 -- Destroying actor for trial train_evaluate_39_lr=0.00011044,momentum=0.99837. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-06-18 17:02:14,494	INFO ray_trial_executor.py:187 -- Destroying actor for trial train_evaluate_38_lr=0.0001807,momentum=0.88554. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-06-18 17:02:14,532	INFO ray_trial_executor.py:187 -- Destroying actor for trial train_evaluate_40_lr=5.9536e-05,momentum=0.99447. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Result for train_evaluate_40_lr=5.9536e-05,momentum=0.99447:
  date: 2019-06-18_17-02-14
  done: false
  experiment_id: ead4174d455a4fc991d56f1f0ebe50a0
  hostname: drfreund-mbp
  iterations_since_restore: 1
  mean_accuracy: 0.9375
  node_ip: 172.31.202.82
  pid: 7289
  time_since_restore: 17.78113579750061
  time_this_iter_s: 17.78113579750061
  time_total_s: 17.78113579750061
  timestamp: 1560891734
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: 4e82a5fa
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/8 CPUs, 0/0 GPUs
Memory usage on this node: 11.4/17.2 GB
Result logdir: /Users/drfreund/ray_results/train_evaluate
Number of trials: 40 ({'TERMINATED': 37, 'RUNNING': 3})
RUNNING trials:
 - train_evaluate_38_lr=0.0001807,momentum=0.88554:	RUNNING
 - train_evaluate_39_lr=0.00011044,momentum=0.99837:	RUNNING
 - train_evaluate_40_lr=5.9536e-05,momentum=0.99447:	RUNNING, [1 CPUs, 0 GPUs], [pid=7289], 17 s, 0 iter, 0.938 acc
TERMINATED trials:
 - tra

[train_evaluate_1_lr=0.0026454,momentum=0.64968,
 train_evaluate_2_lr=0.10476,momentum=0.35651,
 train_evaluate_3_lr=7.7398e-06,momentum=0.78888,
 train_evaluate_4_lr=3.0642e-06,momentum=0.39297,
 train_evaluate_5_lr=0.064553,momentum=0.96158,
 train_evaluate_6_lr=0.0036458,momentum=0.043895,
 train_evaluate_7_lr=0.00027152,momentum=0.59981,
 train_evaluate_8_lr=0.00029511,momentum=0.25307,
 train_evaluate_9_lr=0.01379,momentum=0.82217,
 train_evaluate_10_lr=0.019602,momentum=0.18524,
 train_evaluate_11_lr=1.6402e-06,momentum=0.74164,
 train_evaluate_12_lr=1.6349e-05,momentum=0.06521,
 train_evaluate_13_lr=0.0015501,momentum=0.87419,
 train_evaluate_14_lr=0.0002666,momentum=0.61035,
 train_evaluate_15_lr=0.00015361,momentum=0.5137,
 train_evaluate_16_lr=1.7283e-05,momentum=0.29995,
 train_evaluate_17_lr=3.1434e-05,momentum=0.33509,
 train_evaluate_18_lr=0.00080983,momentum=0.48662,
 train_evaluate_19_lr=0.0010525,momentum=0.54198,
 train_evaluate_20_lr=0.0010544,momentum=0.60231,
 trai

## 5. Retrieve the optimization results

In [8]:
best_parameters, values = ax.get_best_parameters()
best_parameters

{'lr': 0.001305233900549635, 'momentum': 0.6522411436088738}

In [9]:
means, covariances = values
means

{'mean_accuracy': 0.9708333515685448}

## 6. Plot the response surface and optimization trace

In [10]:
render(
    plot_contour(
        model=ax.generation_strategy.model, param_x='lr', param_y='momentum', metric_name='mean_accuracy'
    )
)

In [11]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean * 100 for trial in ax.experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="Accuracy",
)
render(best_objective_plot)